In [1]:
import json
from bs4 import BeautifulSoup
import os
import re
from hugchat import hugchat
from hugchat.login import Login
from hugchat.exceptions import ChatError
import urllib.error

# The path of each catalog
segittur_2022 = "../Catalogs/Segittur/catalog_segittur_2022.json"
segittur_2023 = "../Catalogs/Segittur/catalog_segittur_2023.json"
adestic_v1 = "../Catalogs/Adestic/catalogo_soluciones_turisticas_V1.json"
adestic_v2 = "../Catalogs/Adestic/catalogo_soluciones_turisticas_V2.json"


In [11]:
""" It will transform and reduce the content of the catalog json file to a JSON object where each key is a 
solution name and value is a dictionary with the company, product description, page and solution types. 
In addition, all html tags are removed. This way, it's easier to search for repeats. 
An example: 
"COUNTING PEOPLE ON BUSES USING ARTIFICIAL INTELLIGENCE": {
    "company": "ABACO INGENIERIA Y SEGURIDAD",
    "product_description": "...",
    "page": 10,
    "solType": [
        "Security/Blockchain/Capacity control"
    ]
}
The parameter isAdestic needs to be True when the catalog is from Adestic, to remove the part of the solution name 
"(Original name (Spanish) ...)"
"""
def extract_info_to_analyze_repetitions(file, isAdestic=False):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

        solutions = {}

        for company in data['companies']:
            if re.search('<.*?>', company):
                soup = BeautifulSoup(company, 'html.parser')
                # print(json.dumps(company, indent=4))
                company_name = soup.h2.text
            else:
                company_name = company
                
            if re.search('<.*?>', data['companies'][company]['company_url']):
                company_url= re.sub('<.*?>', '', data['companies'][company]['company_url'])
            else:
                company_url = data['companies'][company]['company_url']

            for solution in data['companies'][company]['products']:
                if re.search('<.*?>', solution['product_name']):
                    soup = BeautifulSoup(solution['product_name'], 'html.parser')
                    solution_name = soup.h1.text
                else:
                    solution_name = solution['product_name']
                    
                if re.search('<.*?>', solution['product_description']):
                    solution_description = re.sub('<.*?>', '', solution['product_description'])

                if isAdestic:
                    solution_name = solution_name.split('(Original name (Spanish)')[0].strip()

                if type(solution['solType']) is list:
                    sol_types = solution['solType']
                else:
                    soup = BeautifulSoup(solution['solType'], 'html.parser')
                    sol_types = [li.text for li in soup.find_all('li')]
                    
                page_number = solution['page']

                solutions[solution_name] = {
                    "company": company_name.strip(),
                    "company_url": company_url.replace("www.",""),
                    "product_description": solution_description,
                    "page": page_number,
                    "solType": sol_types
                }
    with open(f"Small Versions/Small_Version_{os.path.basename(file)}", "w", encoding='utf-8') as f:
        f.write(json.dumps(solutions, indent=4))

In [12]:
# Uncomment the lines below to run the function with each catalog and with isAdestic=True for the Adestic catalogs
# extract_info_to_analyze_repetitions(adestic_v1, True)
# extract_info_to_analyze_repetitions(adestic_v2, True)
# extract_info_to_analyze_repetitions(segittur_2022)
# extract_info_to_analyze_repetitions(segittur_2023)

In [2]:
# The path of each small version of the catalog resulting from the function extract_info_to_analyze_repetitions
small_segittur_2022 = "Small Versions/Small_Version_catalog_segittur_2022.json"
small_segittur_2023 = "Small Versions/Small_Version_catalog_segittur_2023.json"
small_adestic_v1 = "Small Versions/Small_Version_catalogo_soluciones_turisticas_V1.json"
small_adestic_v2 = "Small Versions/Small_Version_catalogo_soluciones_turisticas_V2.json"

In [7]:
""" Some solution types in the json files don't have the correct solution type name. This function will check 
if the solution type exists in the Type_of_Solution_Association (EN).json file (which contains the equivalent 
solution types among the catalogs). If doesn't exist, it will be added to a list of incorrect solution types. 
Then, a manual correction is necessary. For each incorrect type identified you have to associate the 
right type."""
# def check_if_type_exists(catalog, sol_types, sol_type):
#     for t in sol_types:
#         if (isinstance(t[catalog], list) and sol_type in t[catalog]) or (isinstance(t[catalog], str) and sol_type == t[catalog]):
#             return True
#     return False
# 
# def check_for_incorrect_sol_types(catalog, solutions_file):
#     with open(solutions_file, 'r', encoding='utf-8') as f:
#         solutions = json.load(f)
#     with open("Type_of_Solution_Association (EN).json", 'r', encoding='utf-8') as f:
#         sol_types = json.load(f)
#         
#     incorrect_sol_types = []
#         
#     for sol in solutions.keys():
#         for t in solutions[sol]['solType']:
#             # print(f"{t}, {not check_if_type_exists(catalog, sol_types, t)}, {t not in incorrect_sol_types}")
#             if not check_if_type_exists(catalog, sol_types, t) and t not in incorrect_sol_types:
#                 incorrect_sol_types.append(t)
#     
#     with open(f"Small Versions/Incorrect_Sol_Types_{catalog}_Without_Correction.json", "w", encoding='utf-8') as f:
#         # print(f"Writing...\n{json.dumps(incorrect_sol_types, indent=4)}")
#         f.write(json.dumps(incorrect_sol_types, indent=4))
                
""" This function needs to be updated """

In [8]:
# Uncomment the lines below to run the function for each catalog
# check_for_incorrect_sol_types("Segittur 2022", small_segittur_2022)
# check_for_incorrect_sol_types("Segittur 2023", small_segittur_2023)
# check_for_incorrect_sol_types("Adestic V1", small_adestic_v1)
# check_for_incorrect_sol_types("Adestic V2", small_adestic_v2)

In [14]:
# After manually correcting the incorrect solution types, the small versions json file is corrected

def correct_sol_types(catalog, incorrect_types_file):
    with open(catalog, 'r', encoding='utf-8') as f:
        data_catalog = json.load(f)
        
        with open(incorrect_types_file, 'r', encoding='utf-8') as i:
            incorrect_types = json.load(i)
            
            for sol in data_catalog.keys():
                fixed_sol_types = []
                for t in data_catalog[sol]['solType']:
                    if t in incorrect_types:
                        fixed_sol_types.append(incorrect_types[t])
                    else:
                        fixed_sol_types.append(t)
                data_catalog[sol]['solType'] = fixed_sol_types
                        
    with open(catalog, 'w', encoding='utf-8') as f:
        f.write(json.dumps(data_catalog, indent=4))
#         
# correct_sol_types(small_segittur_2022, "Small Versions/Incorrect_Sol_Types_Segittur 2022.json")
# correct_sol_types(small_adestic_v1, "Small Versions/Incorrect_Sol_Types_Adestic V1.json")
# correct_sol_types(small_adestic_v2, "Small Versions/Incorrect_Sol_Types_Adestic V2.json")
#     

In [24]:
# The json objects of the small versions of the catalogs
sol_segittur_2022 = json.load(open(small_segittur_2022, 'r', encoding='utf-8'))
sol_segittur_2023 = json.load(open(small_segittur_2023, 'r', encoding='utf-8'))
sol_adestic_v1 = json.load(open(small_adestic_v1, 'r', encoding='utf-8'))
sol_adestic_v2 = json.load(open(small_adestic_v2, 'r', encoding='utf-8'))

In [4]:
cookie_path_dir = "../cookies"

login_path = "C:\\Users\Diogo Cosme\Documents\ISCTE\Tese\huggingFace login.json"
login_path = login_path.replace("\\", "/") 
login_info = json.load(open(login_path, 'r'))
""" If Login hasn't been done before"""
# sign = Login(login_info['email'], login_info['password'])
# cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)
# cookies_dict = cookies.get_dict()

""" If Login has been done before, replace the path below with your cookies json file path """
cookies_dict = json.load(open(f"{cookie_path_dir}/{login_info['email']}.json", 'r'))

""" This will create a new conversation """
chatbot = hugchat.ChatBot(cookies=cookies_dict)

In [ ]:
def check_if_solutions_were_already_added_to_repetitions(solution1, catalog1, solution2, catalog2, company_url, repeated_solutions):
    solution1_added, solution2_added = False, False
    index = -1
    for i, repetition in enumerate(repeated_solutions):
        if repetition['company_url'] == company_url:
            if catalog1 in repetition and any(sol['solution'] == solution1 for sol in repetition[catalog1]):
                solution1_added = True
            if catalog2 in repetition and any(sol['solution'] == solution2 for sol in repetition[catalog2]):
                solution2_added = True
            if solution1_added or solution2_added:
                index = i
                break
    return solution1_added, solution2_added, index

def create_json_object_to_repeated_solution(name, page):
    return {
        "solution": name,
        "page": page
    }
    
""" Everytime a repeated solution is identified, it will be added to the repeated_solutions dictionary. 
The key is the solution name and the value is a dictionary with the company, the solution page number in 
each catalog, and the method by which it was identified. Either by comparing by solution and company 
names or through LLMs."""
def add_repeated_solution(solution, repeated_solutions, catalog_name, solution_data, other_catalog_name, other_solution_name, other_solution_data, method):
    solution_added, other_solution_added, index = check_if_solutions_were_already_added_to_repetitions(solution, catalog_name, other_solution_name, other_catalog_name, solution_data['company_url'], repeated_solutions)
    
    if method == 'LLM':
        print(f"\n{solution} vs {other_solution_name}\nSolution Added: {solution_added}; Other Solution Added: {other_solution_added}; Index: {index}")
    
    if index == -1:
        repeated_solutions.append({
            "company_url": solution_data['company_url'],
            "methods": [method]
        })
        index = len(repeated_solutions) - 1
    
    if not solution_added:
        solution_json = create_json_object_to_repeated_solution(solution, solution_data['page'])
        if catalog_name not in repeated_solutions[index]:
            repeated_solutions[index][catalog_name] = [solution_json]
        else:
            repeated_solutions[index][catalog_name].append(solution_json)
        
    if not other_solution_added:
        other_solution_json = create_json_object_to_repeated_solution(other_solution_name, other_solution_data['page'])
        if other_catalog_name not in repeated_solutions[index]:
            repeated_solutions[index][other_catalog_name] = [other_solution_json]
        else:
            repeated_solutions[index][other_catalog_name].append(other_solution_json)
    
    if method not in repeated_solutions[index]['methods']:
        repeated_solutions[index]['methods'].append(method)
    return repeated_solutions

"""Everytime a solution is identified as not being a repetition by the LLM, it will be added to the 
false_repetitions dictionary. The key is the solution name and the value is a dictionary with the 
company, the solution page number in the respective catalog, and the other solutions with which it 
was compared. Is only relevant to check that the comparison from LLMs is working correctly."""
def add_false_repetitions(solution_name, solution_data, false_repetition_name, false_repetition_data, false_repetitions, catalog_name, other_catalog_name):
    if solution_name not in false_repetitions:
        false_repetitions[solution_name] = {
            catalog_name: solution_data['page'],
            "others_solutions": [{
                "other_solution": false_repetition_name,
                other_catalog_name: false_repetition_data['page']
            }]
        }
    else:
        # print(json.dumps(false_repetitions[solution_name], indent=4))
        already_added = False
        for other in false_repetitions[solution_name]["others_solutions"]:
            if other["other_solution"] == false_repetition_name and other_catalog_name in other and other[other_catalog_name] == false_repetition_data['page']:
                already_added = True
                break
        if not already_added:
            false_repetitions[solution_name]["others_solutions"].append({
                "other_solution": false_repetition_name,
                other_catalog_name: false_repetition_data['page']
            })
    return false_repetitions


""" If there is a solution in other catalog with the exact same solution and company names, it 
will return True and the solution will be added to the repeated_solutions dictionary. Otherwise, 
it will return False."""
def compare_solution_by_name_and_company(solution, catalog_name, catalog_data, other_catalog_name, other_catalog_data, repeated_solutions):
    it_was_added = True
    if solution in other_catalog_data and catalog_data[solution]['company_url'] == other_catalog_data[solution]['company_url']:
        repeated_solutions = add_repeated_solution(solution, repeated_solutions, catalog_name, catalog_data[solution], other_catalog_name, solution, other_catalog_data[solution], 'Name and Company')
    else:
        it_was_added = False

    return repeated_solutions, it_was_added


def get_not_common_solutions_types(catalog_name, other_catalog_name, types_association):
    not_common_types = []

    for association in types_association:
        if association[catalog_name] == "" and association[other_catalog_name] != "":
            not_common_types.append(association)
    return not_common_types

"""
Getting the solution types association for this specific solution. It will return a dictionary 
with 4 keys (one for each catalog) and the type(s) of the respective catalog. An example for a 
solution from the Segittur 2022 with Other Hardware / Software Solutions type.
{
    "Segittur 2022": "Other Hardware / Software Solutions",
    "Segittur 2023": "Other solutions HW / SW",
    "Adestic V1": [
        "Software",
        "POS Software"
    ],
    "Adestic V2": "Software"
}
If one catalog doesn't have an explicit equivalent solution type, it will return a list of 
dictionaries with the not common solution types. An example: I want solutions with the 
Accessibility type from Segittur 2022. The Segittur 2023 doesn't have an explicit equivalent 
type. 
{
    "Segittur 2022": "Accessibility",
    "Segittur 2023": "",
    "Adestic V1": "Accessibility",
    "Adestic V2": "Accessibility"
}
So, the get_possible_equivalent_solutions_types will return:
{
    "Segittur 2022": "Accessibility",
    "Segittur 2023": [{
        "Segittur 2022": "",
        "Segittur 2023": "Intelligent Signage/Totems/Tourism Signage",
        "Adestic V1": "",
        "Adestic V2": ""
    }, {...}],
    "Adestic V1": "Accessibility",
    "Adestic V2": "Accessibility"
}
The value of Segittur 2023 is all the associations of Segittur 2023 where there is not an 
explicit equivalent solution type with Segittur 2022. This allows me to get all the 
solutions that may be the same, but are in different categories because they are not all 
the same between catalogs.
"""
def get_possible_equivalent_solutions_types(sol_types, catalog_name, types_association):
    for sol_type in sol_types:
        for association in types_association:
            if (isinstance(association[catalog_name], list) and sol_type in association[catalog_name]) or sol_type == association[catalog_name]:
                for key, value in association.items():
                    if value == "":
                        association[key] = get_not_common_solutions_types(catalog_name, key, types_association)
                return association
    return {}

""" Getting solutions from other catalog that have the same company and an equivalent 
solution type"""
def get_solutions_from_the_same_company_and_equivalent_type(company, other_catalog_data, other_catalog_name, sol_types_association, filter_by_type):
    solutions = {}

    # Equivalent solution type of the other catalog from the Type_of_Solution_Association (EN).json file.
    sol_type_association = sol_types_association[other_catalog_name]
    is_list_of_str_association = isinstance(sol_type_association, list) and all(isinstance(item, str) for item in sol_type_association)
    is_list_of_dict_association = isinstance(sol_type_association, list) and all(isinstance(item, dict) for item in sol_type_association)

    for other_solution_key, other_solution_value in other_catalog_data.items():
        # if other_solution_value['company'] == company:
        if other_solution_value['company_url'] == company:
            if filter_by_type:
                # Solution type of the iterated solution from the other catalog
                sol_type = other_solution_value['solType']
                is_list_sol_type = isinstance(sol_type, list)
    
                if is_list_of_str_association and is_list_sol_type and any(element in sol_type_association for element in sol_type):
                    solutions[other_solution_key] = other_solution_value
                elif (is_list_of_str_association and sol_type in sol_type_association) or (is_list_sol_type and sol_type_association in sol_type):
                    solutions[other_solution_key] = other_solution_value
                elif sol_type_association == sol_type:
                    solutions[other_solution_key] = other_solution_value
                elif is_list_of_dict_association:
                    for dict_association in sol_type_association:
                        if dict_association[other_catalog_name] == sol_type:
                            solutions[other_solution_key] = other_solution_value
            else:
                solutions[other_solution_key] = other_solution_value

    return solutions

""" Here, hugchat will be used to compare solutions from the same company and with 
equivalent solution types. If the solution is identified as a repetition, it will 
be added to the repeated_solutions dictionary. If it is identified as not being a 
repetition, it will be added to the false_repetitions dictionary. The aim is for 
LLM to return a json object with the key repetition and with two possible values: 
"YES" or "NO". 
{"repetition": "YES"}
OR 
{"repetition": "NO"}
Please note that there may be unforeseen errors and that LLM may not respond as 
expected. 
In the first case, if this happens, the program ends and the information about 
the solution that was being compared is saved (Execution_Data.json) so that next 
time you can only start comparing from there.  
In the second case, which means that the LLM did not return a JSON object with the 
key repetition and the possible values, this comparison is recorded. Most likely, 
it won't happen, or it won't happen very often. Therefore, if there is a record, 
a manual comparison can be made.
The prompt template is on Prompt_To_Identfy_Repetitions.txt file. An example of 
application is on Prompt_To_Identfy_Repetitions_Example.txt file. 
"""
def compare_descriptions_with_llms(solution_name,solution_data, catalog_name, other_catalog_name, possible_repetitions, repeated_solutions, false_repetitions, wrong_answers):
    print(f"Analysing {len(possible_repetitions.keys())} possible repetitions")
    solution_1 = {
        "title": solution_name,
        "product_description": solution_data['product_description'],
        "company": solution_data['company'],
        "solType": solution_data['solType']
    }
    
    prompt = open("Prompt_To_Identfy_Repetitions.txt", "r", encoding='utf-8').read()
    prompt = prompt.replace("```SOLUTION 1 ´´´", json.dumps(solution_1))
    
    for rep in possible_repetitions:
        print(f"\tSolution: {rep}")
        solution_2 = {
            "title": rep,
            "product_description": possible_repetitions[rep]['product_description'],
            "company": possible_repetitions[rep]['company'],
            "solType": possible_repetitions[rep]['solType']
        }
        new_prompt = prompt.replace("```SOLUTION 2 ´´´", json.dumps(solution_2))
        # Sometimes the hugchat chatbot returns an error ("Exception: Failed to get conversation info with status code: 500")
        try:
            query_result = chatbot.chat(new_prompt)
            print(f"\n\tQuery result: {query_result}")
        except urllib.error.HTTPError:
            print(f"Internal Server Error")
            return None
        except ChatError:
            print(f"Chat Error")
            return None
        except Exception:
            return None
        
        match = re.search(r'\{.*?\}', query_result.text, re.DOTALL)
        if match:
            try:
                yes_or_no = json.loads(match.group())
                # print(f"\t\tJSON OBJECT: {yes_or_no}")
                if yes_or_no['repetition'].lower() == "yes":
                    repeated_solutions = add_repeated_solution(solution_name, repeated_solutions, catalog_name, solution_data, other_catalog_name, rep, possible_repetitions[rep], 'LLM')
                    print(f"Repetition identified by LLM")
                elif yes_or_no['repetition'].lower() == "no":
                    false_repetitions = add_false_repetitions(solution_name, solution_data, rep, possible_repetitions[rep], false_repetitions, catalog_name, other_catalog_name)
                    print(f"Repetition NOT identified by LLM")
            except json.JSONDecodeError:
                print(f"Something went wrong on extracting the JSON object in the query result to Solution 1 {solution_name} and Solution 2 {rep}")
        else:
            wrong_answers.append([solution_1, solution_2])
            print(f"Something went wrong in the query result to Solution 1 {solution_name} and Solution 2 {rep}")
                
    return repeated_solutions, false_repetitions, wrong_answers

        
def save_results(repeated_solutions, false_repetitions, wrong_answers):
    with open("Repeated_Solutions.json", "w", encoding='utf-8') as f:
        f.write(json.dumps(repeated_solutions, indent=4))
    with open("False_Repetitions.json", "w", encoding='utf-8') as f:
        f.write(json.dumps(false_repetitions, indent=4))
    with open("Wrong_Answers.json", "w", encoding='utf-8') as f:
        f.write(json.dumps(wrong_answers, indent=4))
        
def save_execution_data_due_to_error(solution_name, catalog_name, other_catalog_name, repeated_solutions, false_repetitions, wrong_answers):
    save_results(repeated_solutions, false_repetitions, wrong_answers)
    with open("Execution_Data.json", "w", encoding='utf-8') as f:
        f.write(json.dumps({
            "solution_name": solution_name,
            "catalog_name": catalog_name,
            "other_catalog_name": other_catalog_name
        }, indent=4))
        
def load_json_file_if_exists(file, return_if_doesnt_exist):
    if os.path.isfile(file):
        return json.load(open(file, 'r', encoding='utf-8'))
    else:
        return return_if_doesnt_exist
        
def check_for_repeated_solutions(catalogs, filter_by_type=True):
    """ File path of the information about the solution that was being compared (The 
    file only if it happened an error the las time). It contains the solution name, 
    the catalog name and the other catalog name of the last comparison that was 
    being made."""
    execution_data_file_path = "Execution_Data.json"
    
    # If there is already repeated solutions saved, it will be loaded. The same for the false repetitions.
    repeated_solutions = load_json_file_if_exists("Repeated_Solutions.json", [])
    false_repetitions = load_json_file_if_exists("False_Repetitions.json", {})
    
    # If there are wrong answers from the LLM saved, they will be loaded
    wrong_answers = load_json_file_if_exists("Wrong_Answers.json", [])
    
    # Loading the equivalent solution types among the catalogs
    types_association = load_json_file_if_exists("Type_of_Solution_Association (EN).json", {})
    
    # If the last time the program was interrupted by an error
    execution_data = load_json_file_if_exists(execution_data_file_path, {})
    
    has_reach_the_catalog_saved = False
    # Iterate over the catalogs
    for i, catalog in enumerate(catalogs):
        catalog_name, catalog_data = catalog
        
        # Checking if there was an error last time and if so, if the current catalog is the same as the one registered
        if execution_data != {} and execution_data['catalog_name'] == catalog_name:
            has_reach_the_catalog_saved = True
            print(f"Has reached the catalog saved: {catalog_name}")
        elif execution_data != {} and not has_reach_the_catalog_saved:
            continue
        
        if i == len(catalogs) - 1: #All catalogs have been compared
            break
            
        # A list with the remaining catalogs
        others_catalogs = catalogs[i+1:]
        
        has_reach_the_solution_saved = False
        # Iterate over the solutions of the current catalog
        for solution in catalog_data:
            # Checking if there was an error last time and if so, if the current solution is the same as the one registered
            if execution_data != {} and execution_data['catalog_name'] == catalog_name:
                if execution_data['solution_name'] == solution:
                    has_reach_the_solution_saved = True
                    print(f"Has reached the solution saved: {solution}")
                elif not has_reach_the_solution_saved:
                    continue
                    
            print(f"\nAnalysing solution: {solution}, page {catalog_data[solution]['page']} from {catalog_name}")
            sol_types_association = get_possible_equivalent_solutions_types(catalog_data[solution]['solType'], catalog_name, types_association)
            
            has_reach_the_other_catalog_saved = False
            """ If the last time the execution got an error, it is not 
            necessary to compare by name and company the solution saved 
            because it failed in the LLM comparasion which is after. 
            Therefore, when this solution is reached, this variable 
            (does_it_need_to_compare_by_name_and_company) becomes false 
            to avoid this step and then returns to true so that the 
            following solutions perform this step (compare by name and 
            company)"""
            does_it_need_to_compare_by_name_and_company = True
            # Iterate over the other catalogs to find repeated solutions  
            for other_catalog_name, other_catalog_data in others_catalogs:
                # Checking if the conditions match the last execution when it got an error. If it matches, doesn't need to compare by name and company
                if execution_data != {} and execution_data['catalog_name'] == catalog_name and execution_data['solution_name'] == solution:
                    if execution_data['other_catalog_name'] == other_catalog_name:
                        has_reach_the_other_catalog_saved = True
                        print(f"Has reached the other catalog saved: {other_catalog_name}")
                        does_it_need_to_compare_by_name_and_company = False
                    elif not has_reach_the_other_catalog_saved:
                            continue
                            
                if does_it_need_to_compare_by_name_and_company:  
                    # Solutions from the same company and with the same name
                    repeated_solutions, it_was_added = compare_solution_by_name_and_company(solution, catalog_name, catalog_data, other_catalog_name, other_catalog_data, repeated_solutions)
            
                    # Move on to the next catalog if a repeated solution was already found in the current other catalog.
                    if it_was_added:
                        print(f"Solution found in {other_catalog_name}, by comparing name and company")
                        continue
                else:
                    # It was already saved the repetitions by name and company the last time the execution got an Exception. The next solution will need to compare by name and company.
                    does_it_need_to_compare_by_name_and_company = True
                
                """ Solutions from the same company and with compatible solution types. 
                Then descriptions will be compared to see if they are the same."""
                # possible_repetitions = get_solutions_from_the_same_company_and_equivalent_type(catalog_data[solution]['company'], other_catalog_data, other_catalog_name, sol_types_association)
                possible_repetitions = get_solutions_from_the_same_company_and_equivalent_type(catalog_data[solution]['company_url'], other_catalog_data, other_catalog_name, sol_types_association, filter_by_type)
                
                if possible_repetitions != {}:
                    result = compare_descriptions_with_llms(solution, catalog_data[solution], catalog_name, other_catalog_name, possible_repetitions, repeated_solutions, false_repetitions, wrong_answers)
                    if result is None:
                        save_execution_data_due_to_error(solution, catalog_name, other_catalog_name, repeated_solutions, false_repetitions, wrong_answers)
                        return
                    else:
                        repeated_solutions, false_repetitions, wrong_answers = result
                else:
                    print(f"No possible repetitions found in {other_catalog_name}.")
                    # print(f"Possible repetitions from {other_catalog_name} of {solution}, with type {catalog_data[solution]['solType']} in {catalog_name}: {json.dumps(possible_repetitions, indent=4)}")

            # This line should be outside the for loop of catalog_data, but inside allows to record the results after iterating a solution and so I have some results saved when I interrupt the execution.
            save_results(repeated_solutions, false_repetitions, wrong_answers)
        
    # The execution data file can be removed after the execution is finished
    if os.path.isfile(execution_data_file_path): 
        os.remove(execution_data_file_path)
    

In [25]:
#     ("Segittur 2023", sol_segittur_2023)
#     ("Adestic V1", sol_adestic_v1)
#     ("Adestic V2", sol_adestic_v2)
# check_for_repeated_solutions([
#     ("Segittur 2022", sol_segittur_2022),
#     ("Segittur 2023", sol_segittur_2023)
# ])
# check_for_repeated_solutions([
#     ("Segittur 2022", sol_segittur_2022),
#     ("Adestic V1", sol_adestic_v1)
# ], filter_by_type=False)
# check_for_repeated_solutions([
#     ("Segittur 2022", sol_segittur_2022),
#     ("Adestic V2", sol_adestic_v2)
# ], filter_by_type=False)
# check_for_repeated_solutions([
#     ("Segittur 2023", sol_segittur_2023),
#     ("Adestic V1", sol_adestic_v1)
# ], filter_by_type=False)
# check_for_repeated_solutions([
#     ("Segittur 2023", sol_segittur_2023),
#     ("Adestic V2", sol_adestic_v2)
# ], filter_by_type=False)
check_for_repeated_solutions([
    ("Adestic V1", sol_adestic_v1),
    ("Adestic V2", sol_adestic_v2)
])

Has reached the catalog saved: Adestic V1
Has reached the solution saved: Tracking and Digital Antigen Test

Analysing solution: Tracking and Digital Antigen Test, page 26 from Adestic V1
Has reached the other catalog saved: Adestic V2
Analysing 3 possible repetitions
	Solution: Y Ticketing

	Query result:              {
	"repetition": "NO"
}

These are two distinct solutions with different objectives, solution types, and descriptions.
Repetition NOT identified by LLM
	Solution: Yadmin

	Query result:              {
	"repetition": "NO"
}

Based on the provided information, these are two distinct solutions offered by Safety Global. The first solution focuses on tracking and digital antigen tests, while the second solution targets administrative tasks such as appointment scheduling, ticket sales, and capacity management.
Repetition NOT identified by LLM
	Solution: YAforo

	Query result:               {
	"repetition": "NO"
}

These two solutions have different focuses, so they cannot be c

Between catalogs of different producers (SEGITTUR and Adestic) there are the same solutions but classified in different ways:
    - Infotourist Office from SEGITTUR 2023 (pag. 164) is classified as "Other solutions HW / SW"
    - Infotourist Office from Adestic V1 (pag. 13) is classified as "Tourism CRM; Digital marketing; Communication, training or webinar platforms; DTI/Smart City Platform
    
Besides that, the LLM although it was able to recnognise that the solutions are not exactly the same, sometimes it identified as being the same because in general they do the same but with different applications domains.

Examples of that:
    {
        "company_url": "urbiotica.com",
        "methods": [
            "LLM"
        ],
        "Segittur 2022": [
            {
                "solution": "FASTPRK: GUIDED PARKING SYSTEM FOR PARK & RIDE CAR PARKS",
                "page": 245
            },
            {
                "solution": "FASTPRK: GUIDED PARKING SYSTEM FOR TOURIST BUSES (PARK & RIDE)",
                "page": 246
            }
        ],
        "Segittur 2023": [
            {
                "solution": "FASTPRK: GUIDED PARKING SYSTEM FOR PARK & RIDE\u00a0CAR\u00a0PARKS",
                "page": 341
            },
            {
                "solution": "FASTPRK: GUIDED PARKING SYSTEM FOR TOURIST BUSES (PARK\u00a0& RIDE)",
                "page": 342
            }
        ]
    }
    
    {
        "company_url": "unblockthecity.com",
        "methods": [
            "LLM",
            "Name and Company"
        ],
        "Segittur 2022": [
            {
                "solution": "UNBLOCK: THE SMART SIDE OF THE CITY",
                "page": 238
            }
        ],
        "Segittur 2023": [
            {
                "solution": "UNBLOCK: THE SMART SIDE OF THE\u00a0CITY",
                "page": 335
            }
        ],
        "Adestic V1": [
            {
                "solution": "Management for spaces and cities",
                "page": 34
            },
            {
                "solution": "Smart QR on any medium and space",
                "page": 34
            },
            {
                "solution": "Qualified data platform",
                "page": 35
            }
        ],
        "Adestic V2": [
            {
                "solution": "Smart QR on any medium and space",
                "page": 42
            },
            {
                "solution": "Management for spaces and cities",
                "page": 42
            },
            {
                "solution": "Qualified data platform",
                "page": 43
            }
        ]
    }